# 02 — MLP Classification (Dense Only)

`Dense(16→32) → ReLU → Dense(32→10)` with calibrated requantization + corrected bias

In [1]:
from _setup import setup_all, PROJECT_ROOT
setup_all()

   NANO-RUST Setup
✅ Virtualenv: C:\Users\suoya\OneDrive\Documents\nanorust\.venv
✅ numpy
✅ torch
✅ maturin

✅ rustc 1.93.1 (01f6ddf75 2026-02-11)

🔧 CARGO_TARGET_DIR = C:\Users\suoya\.nanorust_target
🔨 Building nano-rust-py (maturin develop --release)...
✅ Build OK (4.9s)
       Finished `release` profile [optimized] target(s) in 0.03s
   📦 Built wheel for CPython 3.13 to C:\Users\suoya\AppData\Local\Temp\.tmpr4m7nE\nano_rust_py-0.1.0-cp313-cp313-win_amd64.whl

🎉 Setup complete! Ready to use.


In [2]:
import numpy as np
import torch
import torch.nn as nn
from nano_rust_utils import quantize_to_i8, quantize_weights, calibrate_model
import nano_rust_py

torch.manual_seed(42)
model = nn.Sequential(
    nn.Linear(16, 32, bias=True),
    nn.ReLU(),
    nn.Linear(32, 10, bias=True),
)
model.eval()
print(model)

q_weights = quantize_weights(model)

torch.manual_seed(123)
input_tensor = torch.randn(1, 16)
q_input, input_scale = quantize_to_i8(input_tensor.numpy().flatten())

requant = calibrate_model(model, input_tensor, q_weights, input_scale)
print('Requant:', {k: (m, s) for k, (m, s, _) in requant.items()})

Sequential(
  (0): Linear(in_features=16, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=10, bias=True)
)
Requant: {'0': (7521837, 31), '2': (13704717, 31)}


In [3]:
with torch.no_grad():
    pytorch_out = model(input_tensor).numpy().flatten()

nano = nano_rust_py.PySequentialModel(input_shape=[16], arena_size=8192)
m0, s0, b0 = requant['0']
nano.add_dense_with_requant(
    q_weights['0']['weights'].flatten().tolist(), b0, m0, s0)
nano.add_relu()
m2, s2, b2 = requant['2']
nano.add_dense_with_requant(
    q_weights['2']['weights'].flatten().tolist(), b2, m2, s2)
nano_out = nano.forward(q_input.tolist())

q_pytorch, _ = quantize_to_i8(pytorch_out)
nano_arr = np.array(nano_out, dtype=np.int8)
diff = np.abs(q_pytorch.astype(np.int32) - nano_arr.astype(np.int32))

print(f'PyTorch (i8):  {q_pytorch.tolist()}')
print(f'NANO-RUST:     {nano_arr.tolist()}')
print(f'Max diff: {np.max(diff)}, Mean diff: {np.mean(diff):.2f}')
print(f'Class: PyTorch={np.argmax(q_pytorch)}, NANO-RUST={np.argmax(nano_arr)}')
print(f'\n{"✅ PASS" if np.max(diff) <= 20 else "❌ FAIL"} (tolerance=20)')

PyTorch (i8):  [-21, 37, 42, -49, 99, -127, -26, 90, 48, -113]
NANO-RUST:     [-21, 38, 43, -51, 97, -128, -28, 90, 46, -114]
Max diff: 2, Mean diff: 1.20
Class: PyTorch=4, NANO-RUST=4

✅ PASS (tolerance=20)
